In [13]:
%load_ext autoreload
%autoreload 2

from train import *
from models import *
from tools import load_data, get_label_dict, get_word_labels

from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

from PIL import Image
from IPython.display import display

import numpy as np
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
X_train, y_train = load_data()
X_val, y_val = load_data("val")
X_test, _ = load_data("test")

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

Loading train data

Finished loading data

Loading val data

Finished loading data

Loading test data

Finished loading data



In [15]:
labels = get_label_dict()
test_images = os.listdir("data/tiny-imagenet-200/test/images/")
words = get_word_labels()

Loading labels

Done

Loading words

Done



In [22]:
# Image preprocessing for the data
datagen = ImageDataGenerator(width_shift_range=0.2,
                             height_shift_range=0.2, 
                             channel_shift_range=115., 
                             fill_mode="reflect", 
                             preprocessing_function=shuffle_channels, 
                             rescale=1./255)

testgen = ImageDataGenerator(rescale=1./255)

datagen.fit(X_train)

testgen.fit(X_val)
testgen.fit(X_test)

data = {"train": (X_train, y_train), 
        "val": (X_val, y_val)}

In [17]:
optim = optimizers.Adam(learning_rate=0.001)

In [18]:
model = AlphaNet()
model_prefix = "models/" + model.name + "/" + model.name
model_path = model_prefix + ".h5"

In [19]:
# Checkpoint callback to only save best model
checkpoint = ModelCheckpoint(model_path, 
                             monitor="val_accuracy", 
                             verbose=1, 
                             save_best_only=True, 
                             mode="auto", 
                             period=1)

cb_list = [checkpoint]

In [20]:
history = train(model_path=model_path, 
                restore=False, 
                epochs=300,
                model=model, 
                optim=optim,
                datagen=datagen,
                data=data, 
                cb_list=cb_list)

Model: "AlphaNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 32)        4736      
_________________________________________________________________
batch_normalization_8 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
spatial_dropout2d_5 (Spatial (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        51264     
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 32, 64)        256

KeyboardInterrupt: 

In [ ]:
show_history(history)

In [ ]:
# This loads the model and makes predictions on the test set
model = load_model(model_path)
output = model.predict_generator(testgen.flow(X_test, shuffle=False))
results = []

for result in output:
    results.append(labels[np.argmax(result)])
    
assert len(results) == len(test_images)
print("Test images predicted:", len(results))

In [ ]:
# This writes the test set predictions to file
with open(model_prefix + " Predictions.txt", "w") as test_file:
    for i in range(len(results)):
        test_file.write(test_images[i] + " " + results[i] + "\n")
    print("Predictions saved at", test_file.name)

In [ ]:
# This shows some test set images and the model's predictions on them
for _ in range(10):
    index = np.random.randint(10000)
    img = Image.fromarray(X_test[index], "RGB").resize(size=(128, 128))
    word_label = words[results[index]]
    display(img)
    print(word_label)

In [ ]:
# This saves an HTML version of the notebook for later viewing
os.system("jupyter nbconvert --to html Training.ipynb")

html_path = model_prefix + ".html"
if path.exists(html_path):
    os.remove(html_path)
os.rename("Training.html", html_path)